# Data Cleaning & Preprocessing

This tutorial will walk you through the steps of data cleaning and preprocessing using a dataset of Airbnb listings in NYC from 2019.

Big thanks to Sumon Singh who created this tutorial.

## Load Libraries

First, import necessary libraries such as:

NumPy: for numerical operations.
Pandas: for data manipulation and analysis.
Scikit-learn: for machine learning tasks, including preprocessing.

In [66]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

## Load The Dataset

Load the Airbnb dataset into a pandas DataFrame.

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/sumony2j/Data_Cleaning_Preprocessing/refs/heads/main/AB_NYC_2019.csv')

Explore the dataset by looking at the first few rows, shape, and information about the columns.



In [ ]:
# Show the shpe of the dataframe


In [ ]:
# have you tried to use .info()?

## Check null values

Check for missing values in each column.

## Drop unnecessary columns

Remove columns that are not relevant to the analysis.

In [ ]:
unnecessary_cols = ['name','host_name','id','host_id','last_review']
# remove columns that are not necessary


## Fill missing values

Fill missing values in the 'reviews_per_month' column using the most frequent value.

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [ ]:
df[['reviews_per_month']]=imputer.fit_transform(df[['reviews_per_month']])

In [ ]:
df['price'].min()

# Data Preprocessing: Scaling and Transforming Skewed Data

In [ ]:
# print the range of the price column



In [10]:
# print the range of the reviews_per_month column

## Why scale data?

Many machine learning models (e.g. linear regression, k-means, PCA) assume that all features contribute equally to the distance or error function.  

However, when variables are on **different ranges**:
- Large-scale features dominate optimization.
- Gradient descent converges slowly or gets stuck.
- Coefficients in regression become hard to interpret.

We need a way to bring all variables to a comparable scale **without distorting relationships**.

---

## Range transformations with pandas

Two simple transformations we can do are **Min-Max** and **Z-score** scaling. Let's compute these manually.


In [ ]:
# use .describe() to have a look at the price and reviews_per_month columns


### Min-max scaling

Compute a Min-Max normalized version of both columns. 
Min-Max scaling transforms features to a fixed range, usually [0, 1]. The formula is:
$$ x_{scaled} = \frac{x - X_{min}}{X_{max} - X_{min}} $$

Where x is the original value, X_min is the minimum value of the feature, and X_max is the maximum value of the feature.


In [ ]:
# Min-Max
df['price_minmax'] = (df['price'] - df['price'].min()) / (df['price'].max() - df['price'].min())
df['reviews_minmax'] = (df['reviews_per_month'] - df['reviews_per_month'].min()) / (df['reviews_per_month'].max() - df['reviews_per_month'].min())


Now, let's look at z-scores. A z-score is defined as: 
$$ z = \frac{x - \mu}{\sigma} $$

where x is the original value, μ is the mean of the feature, and σ is the standard deviation of the feature.

You can interpret a z-score as the number of standard deviations a value is from the mean. It measures the distance of a data point from the mean in units of standard deviations.

In [ ]:
# Z-score
df['price_z'] = (df['price'] - df['price'].mean()) / df['price'].std()
df['reviews_z'] = (df['reviews_per_month'] - df['reviews_per_month'].mean()) / df['reviews_per_month'].std()


### ✏️ Your turn
> Run the code above to compute the min-max and z-score normalized version of both columns. Then:
> - **Check:** If the transformations were successful, the new columns should have ranges roughly [0, 1] (Min-Max) or mean ≈ 0, std ≈ 1 (Z-score).


In [ ]:
# your code here

## Using Scikit-learn scalers

We can replicate the same transformations using the widely popular `scikit-learn` library for machine learning. `scikit-learn` provides consistent tools for scaling as part of preprocessing pipelines.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Define a scaler
scaler_minmax = MinMaxScaler()

# fit parameters to data, then transform it
scaled_minmax = scaler_minmax.fit_transform(df[['price', 'reviews_per_month']])

# add the scaled columns to the dataframe
df[['price_mm', 'reviews_mm']] = scaled_minmax


In [ ]:
scaler_standard = StandardScaler()
scaled_standard = scaler_standard.fit_transform(df[['price', 'reviews_per_month']])
df[['price_std', 'reviews_std']] = scaled_standard

### ✏️ Your turn: transform the data using one of the methods above.
> - import the necessary libraries
> - Load the data 
> - transform the price and reviews_per_month columns using a `RobustScaler()`. 

In [ ]:
# import the necessary libraries
...

# Load the data 
...

# transform the price and reviews_per_month columns using Min-Max and Z-score normalization. Pick the method you prefer.
...

In [13]:
# Compare values in the ['price_minmax','price_mm','price_z','price_std'] columns
...



---

## Visualising distributions
It is difficult to understand variables (and their transformations) just by printing their values. A better approach is to visualise the distribution of values. This allows to visually inspect the distributions and their transformations.




In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(10,4))
# sns.histplot(df['price'], bins=50, ax=axes[0], color='tomato')
# sns.histplot(df['reviews_per_month'], bins=50, ax=axes[1], color='steelblue')
# axes[0].set_title('Price Distribution')
# axes[1].set_title('Reviews per Month Distribution')
# plt.show()

Visualising the distributions showed us something that we could have not guessed by simply looking a the dataframe outputs, namely the data of these two variables is highly skewed. Skewed data can bias statistical models. 

We can compute the skewness of a distribution by using the `skew()` function in pandas.


In [ ]:
df[['price', 'reviews_per_month']]


---

## Why skewness can be problematic

In **linear regression**, **logistic regression**, and **PCA**, we often assume variables are *approximately normal*:

* Highly skewed variables can create *non-linear residuals* and *non-constant variance*.
* They can reduce predictive performance or interpretability.
* They make distance-based metrics unreliable (e.g. k-means centroids get pulled toward long tails).

**Goal:** We want to transform the data to reduce skewness and make the distribution closer to Gaussian.


### Common transformations

| Method                      | Description                                   | When to use                   |
| --------------------------- | --------------------------------------------- | ----------------------------- |
| **Logarithmic**             | `log(x + 1)`                                  | Positive, right-skewed data   |
| **Square root / cube root** | `sqrt(x)` or `x**(1/3)`                       | Count data, moderate skew     |
| **Box-Cox**                 | Power transform for strictly positive data    | Positive data only            |
| **Yeo-Johnson**             | Box-Cox variant that works with negatives     | Any real values               |
| **Quantile transform**      | Maps data to a uniform or normal distribution | Extreme skew or multimodality |

---

## Applying Quantile, Box-Cox, and Yeo-Johnson (using scikit-learn)

Let’s explore how these methods reshape distributions.
We’ll use synthetic data to illustrate the effects (based on the official scikit-learn example).



In [ ]:
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

# Box-Cox (only works if all values > 0)
bc = PowerTransformer(method='box-cox')
df['price_boxcox'] = bc.fit_transform(df[['price']])


Let's visualise the results of the Box-Cox transformation.

In [24]:
# Yeo-Johnson
yj = PowerTransformer(method='yeo-johnson')
df['reviews_yj'] = yj.fit_transform(df[['reviews_per_month']])


Let's visualise the results of the Jeo-Johnson transformation.

Let's visualise the Box-Cox and Yeo-Johnson transformations side by side.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,4))
sns.histplot(df['price_boxcox'], bins=50, ax=axes[0], color='darkorange')
sns.histplot(df['reviews_yj'], bins=50, ax=axes[1], color='teal')
axes[0].set_title('Price after Box-Cox')
axes[1].set_title('Reviews per Month after Yeo-Johnson')
plt.show()


### ✏️ Your turn: Quantile transformation
> A last transform that we can use is the Quantile transformation. This transfrormation maps the data to a uniform or normal distribution, using the rank of the data rather than its actual value. Apply a Quantile transformation to the price or the reviews column and visualise the results. In scikit learn the `QuantileTransformer` class can be used for this purpose.


In [ ]:
# Quantile transform to normal
qt = QuantileTransformer(output_distribution='uniform', random_state=42)
df['price_qt'] = qt.fit_transform(df[['price']])

Let's visualise the results of the Quantile transformation.



---

## Summary

| Problem                       | Transformation                      | Library                                                         |
| ----------------------------- | ----------------------------------- | --------------------------------------------------------------- |
| Variables on different scales | Min-Max / Standardization           | `pandas`, `sklearn.preprocessing`                               |
| Highly skewed distributions   | Log, Box-Cox, Yeo-Johnson, Quantile | `sklearn.preprocessing.PowerTransformer`, `QuantileTransformer` |

---

### Key takeaways

* Scaling makes features comparable and stabilises model training.
* Skewness can break assumptions of linear models.
* Power and quantile transformations can make data more Gaussian.
* Always visualise before and after transforming!


# Encode catagorical variables

Categorical variables are non-numeric variables that represent discrete categories or groups. Examples include sex, color, brand, city, etc. To be used in machine learning models, these categorical variables need to be converted into a numerical format. This process is called encoding.

First we need to identify categorical columns. 

Hint: if the column (or Series) is made of strings, Pandas will assign the 'dtype' method the string 'object'.

In [40]:
# create a list comprehension or a for loop across df columns to get the columns with dtype='object'
l = ...

In [ ]:
l

Check the number of unique values in each categorical column.

In [20]:
print('Number of unique values : ')
# your code here
...

Number of unique values : 


Ellipsis

In [ ]:
df['room_type'].unique()

array(['Private room', 'Entire home/apt', 'Shared room'], dtype=object)

## Convert room_type into numeric data

Convert the 'room_type' column into numerical data using ordinal encoding.

In [41]:
room_type=df['room_type'].to_list()

In [44]:
# convert the room type column into a numpy array
room_type=np.array(room_type)
room_type.shape=(-1,1)

In [45]:
room_type.shape

(48895, 1)

In [ ]:
# Import the ordinal encoder object
from sklearn.preprocessing import OrdinalEncoder
# Create an ordinal encoder object
en=OrdinalEncoder()

# transform room type into an ordinal variable
df['room_type']=en.fit_transform(room_type)

## Convert neighbourhood into numeric data

Convert the 'neighbourhood' column into numerical data using one-hot encoding.

In [52]:
neighbourhood=pd.DataFrame(df['neighbourhood'])

In [ ]:
neighbourhood

,neighbourhood
0,Kensington
1,Midtown
2,Harlem
3,Clinton Hill
4,East Harlem
...,...
48890,Bedford-Stuyvesant
48891,Bushwick
48892,Harlem
48893,Hell's Kitchen


`neighbourhood` is now a dataframe with only one column, also called neighbourhood. The column contains 'strings', basically encoding each neighbourhood with an English word. We want to encode the neighbourhood information in a numerical format, where each neighbourhood is a unique vector. We use the technique known as 'OneHotEncoder' that we have briefly seen in class. 

In [ ]:
encoder=OneHotEncoder(sparse_output=True)
k=encoder.fit_transform(neighbourhood)
k

### ✏️ Your turn 
> Based on your understanding of one hot encoding:
> 1. Does the array contains only zeros? 
> 2. What is the sum of each row of numbers?

In [ ]:
# print all neighbourhoods
encoder.categories_

Let's create a dataframe with each category (neighbourhood) in a separate column and the one hot encoding vectors as separate rows

In [ ]:
p=pd.DataFrame(k,columns=encoder.categories_)

In [ ]:
new_df=pd.concat([df,p],axis=1)

In [ ]:
new_df

## Convert neighbourhood_group into numeric data

Convert the 'neighbourhood_group' column into numerical data using one-hot encoding.

In [ ]:
df['neighbourhood_group'].unique()

In [ ]:
neighbourhood_grp=pd.DataFrame(df['neighbourhood_group'])

### ✏️ Your turn 
> 1. use the one hot encoder to encode `neighbourhood_grp` and assign the result to a variable called `k`
> 2. Use .categories_ to show the categories encoded
> 3. Create a dataframe p using k as the values and categories as column names
> 4. Concatenate new_df and p along the column axis, using `pd.concat`, and assign this new dataframe to `new_df`

In [64]:
# 1. use the one hot encoder to encode `neighbourhood_grp` and assign the result to a variable called `k`
k = ...

In [59]:
# 2. Use .categories_ to show the categories encoded
...

In [63]:
# 3. Create a dataframe p using k as the values and categories as column names
p = ...

In [62]:
# 4. Concatenate new_df and p along the column axis, using `pd.concat`, and assign this new dataframe to `new_df`
new_df=...

In [ ]:
# Display new_df
new_df

Finally, let's drop the original categorical columns.

In [ ]:
new_df.drop(l,axis=1,inplace=True)

In [ ]:
new_df